<a href="https://colab.research.google.com/github/subhashdevarasetty677/Langchain-Q-A-Chatbot/blob/main/Langchainchatbot_Q%26A_with_Groq_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-community langchain streamlit faiss-cpu pypdf transformers groq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [ ]:
!pip install langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.46
    Uninstalling langchain-core-0.3.46:
      Successfully uninstalled langchain-core-0.3.46


In [ ]:
import os
from google.colab import userdata
userdata.get('groq')

os.environ["GROQ_API_KEY"]=userdata.get('groq')
print("API Key Set Successfully!" if "GROQ_API_KEY" in os.environ else "Failed to Set API Key")

API Key Set Successfully!


In [ ]:
%%writefile app.py
import streamlit as st
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS  # ✅ UPDATED
from langchain_community.document_loaders import PyPDFDirectoryLoader  # ✅ UPDATED
from langchain_community.embeddings import HuggingFaceEmbeddings  # ✅ UPDATED
from langchain_groq import ChatGroq # ✅ UPDATED
from langchain.prompts import ChatPromptTemplate

llm = ChatGroq(model_name="llama3-8b-8192")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

prompt = ChatPromptTemplate.from_template(
    """
    please provide the answers based on query
    please provide accurate answers only based on the given prompt
    <context>
    {context}
    </context>
    Question: {input}
    """
)

def vector_embeddings(data_dir = "myfiles1", max_documents=50):
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  loader = PyPDFDirectoryLoader(data_dir)
  documents = loader.load()[:max_documents]
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  texts = text_splitter.split_documents(documents)
  return FAISS.from_documents(texts, embeddings)

vectors = vector_embeddings()

st.title('LangChain Chatbot with Groq API')

input_text = st.text_input('Enter your question')

if input_text:
    document_chain = create_stuff_documents_chain(llm,prompt) #document chain is formed b/n llm and prompt
    retriever = vectors.as_retriever() # we called the vector embedding part
    chain = create_retrieval_chain(retriever,document_chain) #now entire chian pipeline is got built
    response = chain.invoke({"input": input_text})
    st.write(response['answer'])

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.207.111:8501

⠦your url is: https://loud-clouds-fall.loca.lt
/content/app.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
2025-03-24 18:55:48.472755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742842548.496985    3043 cuda_dnn.cc:8310] Unable to register cuDNN factory: 